In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('group_call_HMS.csv')

In [3]:
raw_data.shape 

(6962341, 5)

In [4]:
raw_data.columns

Index(['call_start_day', 'call_start_time', 'call_end_day', 'call_end_time',
       'hashed'],
      dtype='object')

In [5]:
raw_data.head()

,call_start_day,call_start_time,call_end_day,call_end_time,hashed
0,1,09:14:58.558,1,09:41:30.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...
1,17,11:05:05.176,17,13:07:42.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...
2,20,02:18:43.172,20,02:28:58.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...
3,22,09:22:01.936,22,09:47:40.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...
4,26,06:29:21.182,26,06:50:55.004,c87c2fad141edf323f3787335b54be22945a02fe052448...


In [6]:
def change_to_second(x):
    hour = int(x[:2])
    minute = int(x[3:5])
    second = float(x[6:])
    return 3600*hour + 60*minute + second

## 일단 change_to_second 로 초단위로 바꿔준후
## 얼마나 통화했는지로 바꿔보자

In [7]:
raw_data['call_start_time'] = raw_data['call_start_time'].apply(change_to_second)
raw_data['call_end_time'] = raw_data['call_end_time'].apply(change_to_second)

In [8]:
raw_data.head()

,call_start_day,call_start_time,call_end_day,call_end_time,hashed
0,1,33298.558,1,34890.200,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...
1,17,39905.176,17,47262.515,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...
2,20,8323.172,20,8938.177,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...
3,22,33721.936,22,35260.192,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...
4,26,23361.182,26,24655.004,c87c2fad141edf323f3787335b54be22945a02fe052448...


In [9]:
raw_data['call_time']=3600*24*(raw_data['call_end_day']-raw_data['call_start_day'])
                        + raw_data['call_end_time'] - raw_data['call_start_time']





In [10]:
del raw_data['call_start_day']
del raw_data['call_end_day']
del raw_data['call_start_time']
del raw_data['call_end_time']
raw_data.head()

,hashed,call_time
0,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,1591.642
1,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,7357.339
2,86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...,615.005
3,aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...,1538.256
4,c87c2fad141edf323f3787335b54be22945a02fe052448...,1293.822


In [11]:
import tqdm

In [43]:
sum_time =[{},{}]
#sum_count = {}
# generator 생성
temp_hash = (i for i in raw_data['hashed'])
temp_call_time = (i for i in raw_data['call_time'])

In [44]:
# next로 주소값 불러오기 
for i in tqdm.trange(len(raw_data)):
    call_time = next(temp_call_time) # 첫번째 그룹의 call time
    t_h = next(temp_hash) # 첫번째 그룹
    split_hash = t_h.split(',') # [a,b,c,d]
    
    
    temp_split_hash = (user for user in split_hash) #  제너레이터 [a,b,c,d]
    
    for j in range(len(split_hash)):
        user_id = next(temp_split_hash) # 첫번째 그룹의 사람한명
        
        if user_id in sum_time[0].keys():
            sum_time[0][user_id] += call_time
            
        else:
            sum_time[0][user_id] = call_time
            
        if user_id in sum_time[1].keys():
            sum_time[1][user_id] += 1
            
        else:
            sum_time[1][user_id] = 1
        
            
        

100%|██████████| 6962341/6962341 [01:23<00:00, 83593.04it/s]


In [60]:
new_data1 = pd.DataFrame(list(sum_time[0].items()),columns=['ID','TIME'])
new_data2 = pd.DataFrame(list(sum_time[1].items()),columns=['ID','COUNT'])

In [61]:
new_data1.head()

,ID,TIME
0,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,354761.827
1,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583...,78372.345
2,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe8...,448051.062
3,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,855458.296
4,769f00750d87fa59f8b6083aae62fb394e165101796fe5...,774491.333


In [62]:
new_data2.head()

,ID,COUNT
0,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,129
1,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583...,110
2,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe8...,479
3,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,267
4,769f00750d87fa59f8b6083aae62fb394e165101796fe5...,290


In [63]:
new_data1 = pd.merge(new_data1,new_data2,on='ID')
new_data1.head()

,ID,TIME,COUNT
0,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,354761.827,129
1,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583...,78372.345,110
2,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe8...,448051.062,479
3,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,855458.296,267
4,769f00750d87fa59f8b6083aae62fb394e165101796fe5...,774491.333,290
